# 머신러닝 HW1  
- 학번 : 202255535
- 이름 : 김진우

# 1. 데이터 가져오기

UCI 데이터중 heart disease 데이터를 가져와서 classification을 진행해보았다.  
데이터를 가져오는 여러 방법 중 ucimlrepo 라이브러리를 사용하였다.  
또한 필요한 다른 라이브러리들도 함께 import하였다.

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

# 데이터 불러오기
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
heart_disease = fetch_ucirepo(id=45) 
  
# data (as pandas dataframes) 
X = heart_disease.data.features  
y = heart_disease.data.targets   # X를 기반으로 y를 예측하는 것이 목표
  
# metadata 
print(heart_disease.metadata) 
  
# variable information 
print(heart_disease.variables) 


{'uci_id': 45, 'name': 'Heart Disease', 'repository_url': 'https://archive.ics.uci.edu/dataset/45/heart+disease', 'data_url': 'https://archive.ics.uci.edu/static/public/45/data.csv', 'abstract': '4 databases: Cleveland, Hungary, Switzerland, and the VA Long Beach', 'area': 'Health and Medicine', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 303, 'num_features': 13, 'feature_types': ['Categorical', 'Integer', 'Real'], 'demographics': ['Age', 'Sex'], 'target_col': ['num'], 'index_col': None, 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 1989, 'last_updated': 'Fri Nov 03 2023', 'dataset_doi': '10.24432/C52P4X', 'creators': ['Andras Janosi', 'William Steinbrunn', 'Matthias Pfisterer', 'Robert Detrano'], 'intro_paper': {'title': 'International application of a new probability algorithm for the diagnosis of coronary artery disease.', 'authors': 'R. Detrano, A. Jánosi, W. Steinbrunn, M. Pfisterer, J. Schmid, S. Sa

# 2. 데이터 분석

데이터를 train과 test set으로 분리하기 전 전체 데이터(X)에 결측값이 있는지 확인하는 과정을 거쳤다

In [2]:
df = pd.DataFrame(X)

# 각 컬럼별 결측값 수 확인
missing_values = df.isnull().sum()
print(missing_values)

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          4
thal        2
dtype: int64


그 결과 ca와 thal에 NaN값이 존재함을 확인했고, 결측값을 어떻게 채울지 판단하기 위해 각 feature의 데이터 분포를 확인하였다

In [3]:
print(X['ca'].dtype)
print(X['thal'].dtype)

# 데이터 분포 확인
X[['ca', 'thal']].describe()

float64
float64


,ca,thal
count,299.000000,301.000000
mean,0.672241,4.734219
std,0.937438,1.939706
min,0.000000,3.000000
25%,0.000000,3.000000
50%,0.000000,3.000000
75%,1.000000,7.000000
max,3.000000,7.000000


ca는 0에 값이 쏠려있고, thal의 경우 3에 값이 쏠려있음을 확인 할 수있었고, 최빈값으로 값을 채우기로 판단하여 최빈값으로 결측값을 채웠다. 

In [4]:
X['ca'] = X['ca'].fillna(X['ca'].mode().iloc[0])
X['thal'] = X['thal'].fillna(X['thal'].mode().iloc[0])

C:\Users\maure\AppData\Local\Temp\ipykernel_21056\62410547.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['ca'] = X['ca'].fillna(X['ca'].mode().iloc[0])
C:\Users\maure\AppData\Local\Temp\ipykernel_21056\62410547.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['thal'] = X['thal'].fillna(X['thal'].mode().iloc[0])


결측값을 다 채운 후 데이터를 학습 데이터와 테스트 데이터로 분할하는 작업을 수행하였다
 - train_test_split 함수는 주어진 데이터를 무작위로 학습 데이터와 테스트 데이터로 분할해준다

test_size=0.2는 전체 데이터의 20%를 테스트 데이터로 사용하겠다는 것을 의미한다
random_state=42는 데이터를 분할할 때 사용되는 난수 생성기의 시드값으로 동일한 시드값을 사용하면 동일한 데이터 분할을 할 수 있다

In [7]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 정규화
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 3. 모델 선택 및 학습

regression, classification, clustering 중 classification을 사용하였다.  
데이터셋에서 label을 알려주기때문에, unsupervised learning보다는 supervised learning을 선택했다.   
또한 이 dataset은 "presence of heart disease in the patient" 를 구해야하고  
그 정도가 0(없음)부터 4까지의 정수로 표현되는 nominal variable이기 때문에 regression보다는 classification을 쓰는것이 더 적합하다고 판단했다.

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

models = [
    ("Logistic Regression", LogisticRegression(max_iter=1000)),
    ("Decision Tree", DecisionTreeClassifier()),
    ("Random Forest", RandomForestClassifier()),
    ("SVM", SVC()),
    ("K-Nearest Neighbors", KNeighborsClassifier()),
    ("Neural Network", MLPClassifier(max_iter=1000))
]


# 4. 모델 평가

classification을 할 수 있는 여러 모델을 사용하였고, 이 모델들을 평가하는 것으로 F1을 사용하였다

처음에는 accuracy가 가장 직관적인 평가 기준이라고 생각해 accuracy를 사용하여 모델을 평가했지만, 심장병 진단과 같은 의료 분야에서는 잘못된 예측이 치명적일 수 있다고 판단했다.  
예를 들어 심장병이 있는 환자를 심장병이 없다고 예측하는 것과 심장병이 아닌데 심장병이라고 잘못 예측하는 것 모두 큰 문제가 될 수 있기 때문에, 전자의 경우(재현율)와 후자의 경우(정밀도)를 모두 고려하는 F1 점수를 사용하는 것이 가장 적절하다고 판단하였다.  
즉, Recall과 Precision 모두를 고려하는 F1 점수을 모델 평가의 기준으로 삼았다.   

그렇기 때문에 F1 score가 가장 높은 logistic regression 모델을 사용하는 것이 가장 좋다고 판단하였다.

In [10]:
for name, model in models:
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    # accuracy = accuracy_score(y_test, predictions)
    f1 = f1_score(y_test, predictions, average='weighted')  # 'weighted'를 사용하여 불균형 클래스를 고려
    print(f"{name} F1 점수: {f1:.4f}")

c:\Users\maure\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\maure\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\maure\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Logistic Regression F1 점수: 0.5212
Decision Tree F1 점수: 0.4541
Random Forest F1 점수: 0.4696
SVM F1 점수: 0.4609
K-Nearest Neighbors F1 점수: 0.5063


c:\Users\maure\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neighbors\_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\maure\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1102: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Neural Network F1 점수: 0.4736


c:\Users\maure\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
